## Common patterns

#### What's this about Combinators? Sounds like a farming

Think of it as just another name for utility functions.  
Each data type, depending on its semantics, will have some version of these functions. Some will be merely convenience functions for that specific datatype, others will be part of more general patterns. We won't discuss the former since they're just specific constructs that can be looked up in `fp-ts` documentation.  

But the general patters are worth discussing.   
We've already talked about `map`, that's one combinator. And we've seen how `map` kinda always does the same thing, but with slightly different semantics depending on the underlying data type it's operating on.



### Chains? What's next, sticks and stones?

The easiest way to understand `chain` is to look at `Promise.then`. We've mentioned how `.then` offers the same functionality as `map` and also more. It's the "more" part we're interested in:



In [163]:
type PromiseChain = <A, B>(promise: Promise<A>, f: (a: A) => Promise<B>) => Promise<B>

const chain: PromiseChain = (promise, f) => promise.then(f)

So we can see it's just passing in a handler to `.then`, and that the handler generates another promise. In essences, we're `chaining` async operations in sequence!

In [164]:
import * as F from 'fp-ts/function'
import * as P from './lib/promise'
import { request1, request2 } from './mocks/requests'


const responseThen = Promise.resolve()
    .then(request1)
    .then(request2)
const responseChain = F.pipe(
    Promise.resolve(),
    P.chain(request1),
    P.chain(request2)
)

console.log("then:", await responseThen)
console.log("chain:", await responseChain)



request1 arg: undefined
request1 arg: undefined
request2 arg: 1
request2 arg: 1
then: 2
chain: 2


Let's just analyse this a bit deeper, forget about `Promise`s for second, and let's think about this in a more general way:

In [165]:
type T<A> = { /* Some data type, like Promise */ }

type Chain = <A, B>(f: (a: A) => T<B>) => (ma: T<A>) => T<B>

We're _applying a function_ that works on the "inner" value and returns another "wrapped/enhanced" value which is the result of that function.
So for `Promise`s, `chain` allows us to make several sync calls in sequence, with each subsequent call having access to the previous result. Just like in a normal `Promise.then` _chain_!

But `chain` also works for other data types, so what happens with `Option`?

In [166]:
import * as O from 'fp-ts/Option'

type OptionChain = <A, B>(f: (a: A) => O.Option<B>) => (a: O.Option<A>) => O.Option<B>

const optChain: OptionChain = f => ma => O.isNone(ma) ? O.none : f(ma.value) 

const check = optChain((n: number) => O.some(n > 10 ? n + 10: n + 1))

const one = F.pipe(O.some(1), check)
const two = F.pipe(O.some(20), check)
const three = F.pipe(O.none, check)

console.log("one", one)
console.log("two", two)
console.log("three", three)

one { _tag: 'Some', value: 2 }
two { _tag: 'Some', value: 30 }
three { _tag: 'None' }


So we see that if we have a `None`, the the result of `chain` will still be none, otherwise it's whatever is returned from the provided function. This makes sense, if `Option` represents nullable values, `chain`ing some operation on `null` or `undefined` would result in an error unless we propagate the `undefined` value.  
Interestingly, there's a parallel here with `Promise`s. If at any point in a `.then` chain we receive a `Promise.reject`, then the remaining `.then` calls do not run as they no longer make any sense.

> **Tip**
>
> This is *happy* *path* programming! We're let the data type specify the semantics for what happens when something goes "wrong", and then our functions are free to only focus on the case where stuff goes "right". 
> For `Option`, the implementation of `chain` takes care of the `None` case, by passing `None` to every subsequent call. So our functions never have to worry about checking that case.
> For `Promise`, it's the same thing, we never handle the errors in the  `.then/chain` handlers, we need `.catch` for that. 

## There's a new Filter in town

If you've used `Array`s you've used `.filter` (and `map`!). Since `map` can be applied to `Option` and `Promise`, can we do the same with `filter`?

Well, yes of course! we just need to define what it means to `filter` an `Option/Promise`.

Since we use `Option` to represent optional values, it stands to reason that if `Some` value fails some filter function (ie, the callback passed to `filter` function returns `false`), then that `Some` will now represent a `None` case. If it passes the filter, then we just keep the previous value.   


In [167]:
type FilterOption = <A>(f: (a: A) => boolean) => (ma: O.Option<A>) => O.Option<A>


let optFilter: FilterOption = f => ma => {
    //@ts-expect-error
    if (f(ma.value)) return ma  
    return O.none
}

The `@ts-expect-error` is there because we can't really access `.value` without checking if it's a `Some`. So why haven't we done that? 
Well, if it's a `None` from the start, there's no "inner" value to filter, so we just keep it as `None`. And we already have a function that deals with checking the `None` case, it's `chain`!  
If we try to follow the happy path principle, we don't want to pollute the `filter` function implementation which logic that belong to `chain`, so we can instead rewrite `filter` in terms of `chain` 



In [168]:
optFilter = <A>(f: (a: A) => boolean) => (ma: O.Option<A>) => O.chain((a: A) => f(a) ? O.some(a) : O.none)(ma)

[Function (anonymous)]


> **Tip**
>
> Notice how the param `ma` is merely being passed along?  
> Because the functions are curried, when we pass a function `f` to `optFilter` the return type will be a function that takes an `ma: Option<A>` as a param, which is the exact same as what is returned from `O.chain(f)`. 
> That means that the function returned from `O.chain` is all we need as the body of `optFilter`, and we can eliminate the extra redundant arrow function and make `ma` implicit.
>
> This technique is called *eta reduce*

In [169]:
//same as
optFilter = f => O.chain(a => f(a) ? O.some(a) : O.none)

[Function (anonymous)]


So now we can clearly see that `filtering` is merely a `chain` operation with a specific check `f` provided by the caller. 

We've managed to separate concerns, `chain` deals with chaining operations and `filter` filters stuff! Great!

But why go through all this trouble? 
We've been trying to see how these combinators apply to different data types, and now we know that `filter` works for `Array` and `Option`. We also know that we can implement `filter` in terms of `chain` and that chain also works for `Promise`. So it stands to reason we can `filter` `Promise`s *the exact same way!*

In [170]:
type FilterPromise = <A>(f: (a: A) => boolean) => (ma: Promise<A>) => Promise<A>

const promiseFilter: FilterPromise = f => P.chain(a => f(a) ? Promise.resolve(a) : Promise.reject())

And voilá!
Same thing, same pattern, almost same implementation. You may have noticed this pattern by now, that the combinator implementation for different data types is *almost*, *but not quite* the same.

In this case, notice how in both data types we have a boolean check and we "construct" a new data type for the truthy and falsy cases. If only we had generalized implementation of these "constructors" we could completely abstract `filter` away. IF ONLY...

You might have guessed that's not so hard! we totally could, but patience young Padawan! everything in it's own time, we'll get there.  

For now let's take a look at `filter` in action


In [171]:
const positive = O.some(1)
const negative = O.some(-1)

const positivePromise = Promise.resolve(1)
const negativePromise = Promise.resolve(-1)

const isPositive = (x: number) => x > 0

console.log(F.pipe(positive, O.filter(isPositive)))
console.log(F.pipe(negative, O.filter(isPositive)))
console.log(await F.pipe(positivePromise, P.filter(isPositive)))
// catching just so we can log it
console.log(await F.pipe(negativePromise, P.filter(isPositive)).catch(() => "caught rejection"))


{ _tag: 'Some', value: 1 }
{ _tag: 'None' }
1
caught rejection


## Behold! A `Fold`, the old gold! 

So far we've seen way to interact and operate with the "inner" value of these data types. What about when we want to "extract" it. In other others, get rid of the "container". 

Before we do that, tough. Let's just address some terminology

> We call the "inner" values *monadic*. That's a scary world, but we'll soon explain it as well. For now, just remember, *monadic* means the inner value of the container data type

Now onto `fold`. Continuing with the `Array` methods, we've seen `map` and `filter`. You might have guessed, the next one in line is `reduce`! 

> *Aside*: You maybe have wondered about `chain`? Is there a common method for `Array` that is a parallel of `chain`? Well, yes! See if you can find it! We'll talk more about it later.

`reducing` is a `fold` operation! What does `reduce` do? it takes an `Array` and turns it into a single value. In other words, it gets rid of the `Array` "container". 

Let's look at an example:

In [172]:
const sum = (arr: Array<number>) => arr.reduce((total, n) => n + total, 0)

sum([1,1,1,1,1])

5


`sum` is a `fold`, it transforms an `Array` into a `number`, but it does that by taking into account all the monadic values of the `Array`.

So how does this translate into `Option`? 
Well, `Option` only has one monadic value, but it does represent two different states: `None` and `Some`. So if we want to `fold` an `Option`, we have to provide handlers for both cases:

In [173]:
type OptionFold = <A, B>(onNone: B, onSome: (a: A) => B) => (ma: O.Option<A>) => B

const optFold: OptionFold = (onNone, onSome) => ma => O.isNone(ma) ? onNone : onSome(ma.value)

const extract = optFold<number, string>("None", n => JSON.stringify(n))

console.log(extract(O.some(1)))
console.log(extract(O.none))

1
None


Cool! And now for `Promise`!

In [174]:
type PromiseFold = <A, B>(onRejection: B, onFulfilled: (a: A) => B) => (ma: Promise<A>) => B

const promiseFold: PromiseFold = (onRejection, onFulfilled) => ma => "oh bollocks!" as any

yeah, bit of a bummer there! turns out there's no implementation for the `PromiseFold` type!
If we call `.then` or `.catch` we get back another `Promise`! And remember that `await` is merely syntactic sugar, underneath it's still using `Promise`s, so we can never "extract" the monadic value from a `Promise`!

**This is not a problem, rather, it's a feature!**  
Recall that we're doing async programming here, so we don't know when that code will run, if it will run, or when it will complete. So we can't access the result willy nilly, we always have to specify that we want to do something with the result only when it's resolved. Only `.then` can we continue.  
We can never mix async with synchronous code, which is why it's useful to have the `Promise` type propagated up the call stack!  

So what do we learn from this? 
Turns out these functions are general patterns that apply across different data types, but not always!!  

Sometimes, **it's important to propagate the data type instead of trying to "get rid of them"**.  
This has massive implications for codebase structure.  
If we keep "pushing out" the types, eventually they will propagate up to the top level (think `main`) function/module. There, we can define what the semantics are, so if we want slightly different behaviours for nullability and async, we can just adjust the implementation of `Promise` and `Option`, __*and then everything will still type check and work*__!  
We go from having individual functions dealing with all these effects, to _**having the application react to them as a whole**_, with each function _**only caring about the happy path**_


## Either madness or genius

We're finally ready to talk about `Either`!

`Either` is a slightly different beast as it has two type arguments. We define it as:

In [175]:
type Left<E> = { _tag: "Left", left: E }
type Right<A> = { _tag: "Right", right: A }

type Either<E, A> = Left<E> | Right<A>

> **Tip** Typically, the `Left` side is used for error types, hence the abbreviation `E`.  

Let's think about what it means to `map`, `chain`, `filter` and `fold` an `Either`

The easiest one is probably `fold`.

After all, all we're talking about is checking if we have a correct `Right` value or a `Left` error.


In [176]:
import * as E from 'fp-ts/Either'
type EitherFold = <E, A, B>(onLeft: (e: E) => B, onRight: (a: A) => B) => (ma: E.Either<E, A>) => B

const eFold: EitherFold = (onLeft, onRight) => ma => E.isLeft(ma) ? onLeft(ma.left) : onRight(ma.right)
 
const extract = E.fold<string, number, string>(e => e, n => JSON.stringify(n))

console.log(F.pipe(E.right(10), extract))
console.log(F.pipe(E.left("error"), extract))

4:63 - Argument of type 'Either<E, A>' is not assignable to parameter of type 'Either<unknown, unknown>'.
4:63 - Type 'Left<E>' is not assignable to type 'Either<unknown, unknown>'.
4:63 - Property 'left' is missing in type 'Left<E>' but required in type 'Left<unknown>'.
4:96 - Argument of type 'E | A' is not assignable to parameter of type 'A'.
4:96 - 'A' could be instantiated with an arbitrary type which could be unrelated to 'E | A'.


Cool! Easy!

> *Aside* `E.fold` forces the return type for the `Left` and `Right` side to be the same. There are other similar functions in the `Either` module that relax that restriction


Let's look at `filter` next

If we have a `Right` value, and it fails to pass a `filter`, we're probably talking about an error we want to track. So we should change our `Right` to `Left`

In [ ]:
type FilterEither = <A, E>(f: (a: A) => boolean, onFail: E) => (ma: Either<E, A>) => Either<E, A>

const eitherFilter: FilterEither = (f, e) => E.chain(a => f(a) ? E.right(a) : E.left(e))

So same thing as for `Option`, except we pass in the error value we would like to keep if the `filter` fails.  
As before, we leverage the `chain` mechanics for the filter implementation!

So what are those mechanics? How does `chain` work?

Le's compare with `Option`. There we said that if the value is `None`, then there's no point in chaining anything else. The same could be applied here, if we have an error, we want don't want to do anything else to the value except propagate that error.  
In effect, it's a "bail-on-first-error" kinda thing!

In [ ]:

type EitherChain = <Err, A, B>(f: (a: A) => E.Either<Err, B>) => (a: E.Either<Err, A>) => E.Either<Err, B> // Err for disambiguating with import E

const eChain: EitherChain = f => ma => E.isLeft(ma) ? ma : f(ma.right) 

And now we can test in combination with `filter`

In [ ]:
const rightPos = E.right(10)
const rightNeg = E.right(-10)
const left = E.left("booooo!")

const check =  eitherFilter(isPositive, "not positive")

console.log(check(rightPos))
console.log(check(rightNeg))
console.log(check(left))


So what about `map`? That seemed to be the simplest one before, why haven't we implemented it yet? 

Well, let's write the type of a general `map` function

In [ ]:
type Mapper = <A, B>(f: (a: A) => B) => (a: A) => B

Notice how we only have 2 types? But for `Either`, there's 3: `A`, `B` and the extra `E` for the error type

So how do we deal with this? should we `map` the `Right` or the `Left` side, or both?
Logically, it makes sense to only `map` the `Right`. After all, we said we're implementing "bail-on-first-error".  
We can't do both sides since `E` and `A` rare different types. Besides, we can easily implement another function `bimap`, similar to `map`, which would have 2 mapping functions as params: one for mapping the `Left` side, another for the `Right`.

So we want to `map` only on the `Right`. 
Let's do that!

In [ ]:
type EitherMap = <A, B>(f: (a: A) => B) => <Err>(ma: E.Either<Err, A>) => E.Either<Err, B>

const eMap = f => ma => E.isLeft(ma) ? ma : E.right(f(ma.right))

const stringify = eMap((x: any) => JSON.stringify(x))

console.log(stringify(E.right(1)))
console.log(stringify(E.left("error")))

So what's the big deal? It's not that different from the other data types!

Let's look look closer, `map` only operates on `A`, it's not really aware of `E`. In fact, to provide the mapping function, `E` is not needed!
All the other combinators had to be aware of `E`, but not `map`!  

> **Note** this is why, by convention, we specify the `E` type argument first, so that `map` only works on the right-most (last) type argument

This results in an interesting property.  
Look at the `stringify` function. We can view that as a mapping function *specialized for the `Either` data type*. And we achieved this specialization, by calling the implementation of `map` for `Either`.  


In [ ]:
type GenericMapper = <A, B>(a: A) => B
type EitherMapper = <A, B, E>(a: Either<E, A>) => Either<E, B> 

We turned `GenericMapper` into `EitherMapper` by calling the implementation of `map` for `Either`

This begs the question, does this work for other data types? Well, YES! We've already seen it!

In [ ]:
type OptionMapper = <A, B>(a: O.Option< A>) => O.Option<B> 
type PromiseMapper = <A, B>(a: Promise<A>) => Promise<B> 

If we call the implementation of map for `Option`, we get an `OptionMapper`  
If we call the implementation of map for `Promise`, we get an `PromiseMapper`  


We can generalize `map` to that it works on "containers". Doing so means that calling any implementation of `map` for a particular data type *lifts* (ie, specializes) the mapping function to a version which works on the *monadic* values of the specified "container" (as in, `Option`, `Either`, `Promise`, etc) 

In [ ]:
type Container<A> = any /* some data type*/ 

type ContainerMap = <A, B>(f: (a: A) => B) => (a: Container<A>) => Container<B>

Now, the type signature above might be elucidative, but it's actually not easy to implement in TypeScript. And we'll run into problems when working with `Either`
We'll see why in a subsequent chapter, where we will abstract these combinators so we can work with them generically, without specializing it to each data type

For now, the important thing to note are:  

* We can use the same patterns and functions for each data type
* The combinators aways implement the same pattern
* Each data type will have different semantics
* `map` *lifts* a function to a "container"
* `Either` has 2 type arguments, making life slightly different